<a href="https://colab.research.google.com/github/mahalakshmijinadoss/pooling/blob/main/trac2_taskb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece]==4.10.0 datasets --quiet


     |████████████████████████████████| 2.8 MB 5.4 MB/s 
     |████████████████████████████████| 311 kB 48.3 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 29.4 MB/s 
     |████████████████████████████████| 895 kB 45.8 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 1.1 MB 42.4 MB/s 
     |████████████████████████████████| 1.1 MB 51.1 MB/s 
     |████████████████████████████████| 133 kB 43.6 MB/s 
     |████████████████████████████████| 243 kB 47.6 MB/s 
     |████████████████████████████████| 271 kB 47.6 MB/s 
     |████████████████████████████████| 144 kB 50.6 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 


In [2]:
!python -c  "import transformers;print(transformers.__version__)"


4.10.0


In [3]:
!pip install -qq ipython-autotime

%load_ext autotime

time: 243 µs (started: 2022-02-09 15:20:00 +00:00)


In [4]:
import numpy as np
import torch
import random
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

time: 1.73 s (started: 2022-02-09 15:20:12 +00:00)


In [5]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

time: 6.54 s (started: 2022-02-09 15:20:18 +00:00)


In [6]:
import pandas as pd
import numpy as np


time: 1.32 ms (started: 2022-02-09 15:20:29 +00:00)


In [7]:
from transformers import DataCollatorWithPadding


time: 478 ms (started: 2022-02-09 15:20:32 +00:00)


In [8]:
!pip install wandb --quiet

import wandb
wandb.login()


     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 180 kB 48.3 MB/s 
     |████████████████████████████████| 143 kB 48.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

time: 14.7 s (started: 2022-02-09 15:20:34 +00:00)


In [9]:
%env WANDB_PROJECT=OGBV_task_trac2

env: WANDB_PROJECT=OGBV_task_trac2
time: 3.43 ms (started: 2022-02-09 15:20:54 +00:00)


In [10]:
!huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "rohanrajpal/bert-base-en-hi-codemix-cased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/832 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

time: 2.5 s (started: 2022-02-09 15:21:14 +00:00)


In [12]:
trainb= pd.read_csv('/content/trac2_hin_train.csv',sep=',')

time: 53.4 ms (started: 2022-02-09 15:21:25 +00:00)


In [13]:
trainb.head()


,ID,Text,Sub-task A,Sub-task B
0,C4.131,Bollywood film dekhne ke samay logic ghar mein...,NAG,NGEN
1,C4.638,Chutiya movie...,NAG,NGEN
2,C38.598,Us jaat bnde ka khene ka matlab tha mar daluga...,OAG,NGEN
3,C4.2101.1,@Feminism Is CANCER *un feminist yeh sahi hai ...,OAG,NGEN
4,C29.14.2,Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने,NAG,NGEN


time: 79.3 ms (started: 2022-02-09 15:21:27 +00:00)


In [14]:
trainb = trainb.iloc[:,[0,1,3]]


time: 9.51 ms (started: 2022-02-09 15:21:30 +00:00)


In [15]:
trainb.head()


,ID,Text,Sub-task B
0,C4.131,Bollywood film dekhne ke samay logic ghar mein...,NGEN
1,C4.638,Chutiya movie...,NGEN
2,C38.598,Us jaat bnde ka khene ka matlab tha mar daluga...,NGEN
3,C4.2101.1,@Feminism Is CANCER *un feminist yeh sahi hai ...,NGEN
4,C29.14.2,Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने,NGEN


time: 38.5 ms (started: 2022-02-09 15:21:32 +00:00)


In [16]:
trainb.rename(columns = {'Sub-task B':'label'}, inplace = True)


time: 2.65 ms (started: 2022-02-09 15:21:34 +00:00)


In [17]:
trainb.loc[(trainb.label == 'NGEN'), 'label'] = 0
trainb.loc[(trainb.label == 'GEN'), 'label'] = 1

time: 12.3 ms (started: 2022-02-09 15:21:36 +00:00)


In [18]:
devb = pd.read_csv('/content/trac2_hin_dev.csv')


time: 19.5 ms (started: 2022-02-09 15:21:47 +00:00)


In [19]:
devb = devb.iloc[:,[0,1,3]]
devb.rename(columns = {'Sub-task B':'label'}, inplace = True)
devb.loc[(devb.label == 'NGEN'), 'label'] = 0
devb.loc[(devb.label == 'GEN'), 'label'] = 1

time: 13.6 ms (started: 2022-02-09 15:21:50 +00:00)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer

In [20]:
train_texts = list(trainb['Text'])
train_labels = list(trainb['label'])
labels = list(trainb['label'].unique())
num_labels = len(labels)

time: 8.82 ms (started: 2022-02-09 15:21:54 +00:00)


In [21]:
dev_texts = list(devb['Text'])
dev_labels = list(devb['label'])

time: 2.2 ms (started: 2022-02-09 15:21:56 +00:00)


In [22]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

train_encodings = tokenizer(train_texts,truncation=True)
dev_encodings = tokenizer(dev_texts,truncation=True)

time: 1.97 s (started: 2022-02-09 15:21:57 +00:00)


In [23]:
import torch

class TaskDatasetB(torch.utils.data.Dataset):

  def __init__(self,encodings,labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

time: 9.72 ms (started: 2022-02-09 15:22:02 +00:00)


In [24]:
train_dataset = TaskDatasetB(train_encodings,train_labels)
dev_dataset = TaskDatasetB(dev_encodings,dev_labels)

time: 1.56 ms (started: 2022-02-09 15:22:05 +00:00)


In [25]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


time: 982 µs (started: 2022-02-09 15:22:07 +00:00)


In [26]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    'OGBV-gender-bert-hi-en-trac2',
    report_to = "wandb",
    run_name = "rr-bert-base-hi-en-3epoch",
    evaluation_strategy = 'epoch',
    save_strategy='epoch',
    num_train_epochs = 3,
    load_best_model_at_end = True,
    push_to_hub='OGBV-gender-bert-hi-en-trac2'
)

time: 78.3 ms (started: 2022-02-09 15:22:12 +00:00)


In [27]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = num_labels,ignore_mismatched_sizes=True)


Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at rohanrajpal/bert-base-en-hi-codemix-cased and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 29.5 s (started: 2022-02-09 15:22:16 +00:00)


In [28]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

def compute_metrics(eval_preds):

  logits,labels = eval_preds
  predictions  = np.argmax(logits,axis=-1)

  acc = accuracy_score(labels,predictions)
  precision = precision_score(labels,predictions,average='weighted')
  recall = recall_score(labels,predictions,average='weighted')
  f1 = f1_score(labels,predictions,average='weighted')
  
  return {
     'acc' : acc,
     'precision' : precision,
     'recall' : recall,
     'f1' : f1
  }

time: 7.21 ms (started: 2022-02-09 15:23:12 +00:00)


In [29]:
!apt-get install git-lfs


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [30]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,

    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/Maha/OGBV-gender-bert-hi-en-trac2 into local empty directory.


time: 19.3 s (started: 2022-02-09 15:23:27 +00:00)


In [31]:
trainer.train()


***** Running training *****
  Num examples = 3984
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1494
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mahaj (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss,Acc,Precision,Recall,F1
1,No log,0.432343,0.847543,0.718329,0.847543,0.777604
2,0.461400,0.427800,0.847543,0.718329,0.847543,0.777604
3,0.454500,0.427270,0.847543,0.718329,0.847543,0.777604


***** Running Evaluation *****
  Num examples = 997
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to OGBV-gender-bert-hi-en-trac2/checkpoint-498
Configuration saved in OGBV-gender-bert-hi-en-trac2/checkpoint-498/config.json
Model weights saved in OGBV-gender-bert-hi-en-trac2/checkpoint-498/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 997
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to OGBV

TrainOutput(global_step=1494, training_loss=0.4559481415244311, metrics={'train_runtime': 663.7679, 'train_samples_per_second': 18.006, 'train_steps_per_second': 2.251, 'total_flos': 593981429695680.0, 'train_loss': 0.4559481415244311, 'epoch': 3.0})

time: 11min 3s (started: 2022-02-09 15:24:02 +00:00)


In [32]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 997
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epoch': 3.0,
 'eval_acc': 0.847542627883651,
 'eval_f1': 0.7776042568530782,
 'eval_loss': 0.42726996541023254,
 'eval_precision': 0.7183285060799249,
 'eval_recall': 0.847542627883651,
 'eval_runtime': 14.2442,
 'eval_samples_per_second': 69.993,
 'eval_steps_per_second': 8.775}

time: 14.3 s (started: 2022-02-09 15:35:10 +00:00)


In [33]:
model_path = 'OGBV-gender-bert-hi-en-trac2'


time: 4.3 ms (started: 2022-02-09 15:35:27 +00:00)


In [34]:
model.push_to_hub('OGBV-gender-bert-hi-en-trac2')


Configuration saved in OGBV-gender-bert-hi-en-trac2/config.json
Model weights saved in OGBV-gender-bert-hi-en-trac2/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.38k/679M [00:00<?, ?B/s]

To https://huggingface.co/Maha/OGBV-gender-bert-hi-en-trac2
   f0c7159..6f216ea  main -> main



'https://huggingface.co/Maha/OGBV-gender-bert-hi-en-trac2/commit/6f216ea410dace8fecc501934d4adbce56f4d68c'

time: 11min 34s (started: 2022-02-09 15:35:30 +00:00)


In [35]:
tokenizer.push_to_hub('OGBV-gender-bert-hi-en-trac2')


tokenizer config file saved in OGBV-gender-bert-hi-en-trac2/tokenizer_config.json
Special tokens file saved in OGBV-gender-bert-hi-en-trac2/special_tokens_map.json
To https://huggingface.co/Maha/OGBV-gender-bert-hi-en-trac2
   6f216ea..cd383e9  main -> main



'https://huggingface.co/Maha/OGBV-gender-bert-hi-en-trac2/commit/cd383e9e8097f0e1c43e968cdebc19ef5aab5373'

time: 7.47 s (started: 2022-02-09 15:47:10 +00:00)
